In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk

make sure to upload valueset and fhir bundle everytime your runtime shuts down

In [ ]:
df = pd.read_excel('/content/ValueSets (1).xlsx', sheet_name=1)

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

Upload and show RXnorm medication codes for OUR reference

In [ ]:
df_Rx = df[df['Code System'].str.contains('RXNORM') == True]
#df_L[df_L['Code System'] == 'LOINC']
df_Rx

,Name,Code System,Type,Steward,OID,Expansion Status,Expansion Date,Code Count
59,Abemaciclib,RXNORM,Extensional,CancerLinQ,2.16.840.1.113762.1.4.1260.476,Active,2025-09-16,24
87,Abortion Medications,RXNORM,Grouping,NACHC,2.16.840.1.113762.1.4.1221.179,Active,2025-09-16,123
141,Acamprosate,RXNORM,Extensional,NCQA,2.16.840.1.113883.3.464.1003.1154,Not Maintained,2025-09-16,1
153,ACE ARB Prescription,RXNORM,Extensional,Cliniwiz,2.16.840.1.113762.1.4.1200.245,Not Maintained,2025-09-16,284
155,ACE inhibitor allergy RXNORM,RXNORM,Extensional,Optum,2.16.840.1.113762.1.4.1078.45,Active,2025-09-16,279
...,...,...,...,...,...,...,...,...
17040,Zileuton Oral Preparations,RXNORM,Extensional,NHLBI Implementation Science and Health Care I...,2.16.840.1.113762.1.4.1196.468,Not Maintained,2025-09-16,3
17041,Zinc,RXNORM,Extensional,Clinical Architecture,2.16.840.1.113883.3.3616.200.110.102.2091,Deprecated,2023-11-20,65
17042,Ziprasidone,RXNORM,Extensional,NCQA,2.16.840.1.113883.3.464.1003.1141,Not Maintained,2025-09-16,5
17043,Zolmitriptan,RXNORM,Extensional,AAN,2.16.840.1.113762.1.4.1034.656,Active,2025-09-16,4


Upload BioBert for semantic vectors

In [ ]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, models

# 2) Build a Sentence-Transformers pipeline around Bio_ClinicalBERT
#    - 'Transformer' loads the HF model
#    - 'Pooling' does mean-pooling over token embeddings
biobert_name = "emilyalsentzer/Bio_ClinicalBERT"
word_model = models.Transformer(biobert_name, max_seq_length=256)
pooling    = models.Pooling(word_model.get_word_embedding_dimension(),
                            pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[word_model, pooling])

Import FHIR bundles and turn then into python dicts

In [ ]:
import glob
import json

json_dicts = []  # this will store all JSON files as Python dicts

for path in glob.glob("/content/synthetic data/*.json"): # Corrected path pattern
    with open(path, "r") as f:
        data = json.load(f)   # ← this becomes a dict
        json_dicts.append(data)

# Check results
print(f"Loaded {len(json_dicts)} JSON files.")
if json_dicts: # Added a check to ensure the list is not empty before accessing index 0
    print(type(json_dicts[0]))    # should be <class 'dict'>


Loaded 99 JSON files.
<class 'dict'>


function to turn dates into age

In [ ]:
from datetime import datetime, date

def birthday_to_age(birthdate_str):
    # Expecting format "YYYY-MM-DD"
    birthdate = datetime.strptime(birthdate_str, "%Y-%m-%d").date()
    today = date.today()

    age = today.year - birthdate.year - (
        (today.month, today.day) < (birthdate.month, birthdate.day)
    )
    return age

print(birthday_to_age("1990-04-23"))


35


Reference for extracting patient info

In [ ]:
print(json_dicts[0]['entry'][0]['resource']['birthDate'])

1964-01-12


Creating dataset, keep it anonymous to "remain compliant"

In [ ]:
data = pd.DataFrame(columns=['sex', "age", 'weight'])

Weight lives in the observation resource, medical data is really messy so it could be inside any observation, there are likely tons of them. There might be none, we will pick the latest one.

In [ ]:
from datetime import datetime

# weight is in KGs

def get_latest_weight(bundle):
    """
    Given a FHIR Bundle (as a Python dict), return the most recent
    body weight value (int/float) or None if no weight is found.
    """
    weights = []

    # Collect all weight observations with their dates
    for entry in bundle.get("entry", []):
        resource = entry.get("resource", {})

        if resource.get("resourceType") == "Observation":
            # Check if code matches body weight (LOINC 29463-7 or 3141-9)
            codings = resource.get("code", {}).get("coding", [])
            if not any(c.get("code") in ["29463-7", "3141-9"] for c in codings):
                continue

            # measurement value
            value = resource.get("valueQuantity", {}).get("value")

            # date of measurement
            date = (
                resource.get("effectiveDateTime")
                or resource.get("effectivePeriod", {}).get("start")
            )

            if value is not None and date is not None:
                weights.append((value, date))

    # No weights found
    if not weights:
        return None

    # Helper to parse ISO datetimes like "2021-02-05T00:00:00Z"
    def parse_dt(d):
        return datetime.fromisoformat(d.replace("Z", "+00:00"))

    # Pick the (value, date) pair with the latest date
    latest_value, _latest_date = max(weights, key=lambda x: parse_dt(x[1]))

    # Ensure numeric if possible, but keep int/float "as it came"
    if isinstance(latest_value, (int, float)):
        return latest_value
    try:
        return float(latest_value)
    except (TypeError, ValueError):
        return latest_value  # fallback: return as-is


get_latest_weight(json_dicts[0])

103.6

In [80]:
# Initialize lists
sex_list = []
age_list = []
weight_list = []
frequency_list = []
period_list = []
periodUnit_list = []
asNeeded_list = []
code_list = []

for patient_bundle in json_dicts:
    # Extract patient demographics once per patient bundle
    patient_resource = patient_bundle['entry'][0]['resource']
    patient_sex = patient_resource['gender']
    patient_age = birthday_to_age(patient_resource['birthDate'])
    patient_weight = get_latest_weight(patient_bundle)

    # Iterate through entries to find MedicationRequests
    for entry in patient_bundle['entry']:
        resource = entry.get('resource', {})
        if resource.get('resourceType') == "MedicationRequest":
            dosage_instructions = resource.get('dosageInstruction')

            if dosage_instructions and len(dosage_instructions) > 0:
                dosage_instruction = dosage_instructions[0]

                # Extract asNeededBoolean early
                current_asNeeded = dosage_instruction.get('asNeededBoolean')

                # Extract timing info
                timing = dosage_instruction.get('timing', {})
                repeat = timing.get('repeat', {})

                current_frequency = repeat.get('frequency')   # may be None
                current_period = repeat.get('period')         # may be None
                current_periodUnit = repeat.get('periodUnit') # may be None

                # Extract medication code
                med_code = None
                med_cc = resource.get('medicationCodeableConcept')
                if med_cc:
                    coding = med_cc.get('coding')
                    if coding and len(coding) > 0:
                        med_code = coding[0].get('code')

                # Skip if no code (optional — remove if you want rows without codes)
                if med_code is None:
                    continue

                # -----------------------------
                # CASE 1: NORMAL DOSING VALUES
                # -----------------------------
                if (current_frequency is not None and
                    current_period is not None and
                    current_periodUnit is not None):

                    sex_list.append(0 if patient_sex == 'female' else 1)
                    age_list.append(patient_age)
                    weight_list.append(patient_weight)
                    frequency_list.append(current_frequency)
                    period_list.append(current_period)
                    periodUnit_list.append(current_periodUnit)
                    asNeeded_list.append(current_asNeeded if current_asNeeded is not None else False)
                    code_list.append(med_code)

                # -----------------------------------------------
                # CASE 2: asNeeded=True AND no frequency info
                # → Fill frequency/period/periodUnit with -1
                # -----------------------------------------------
                elif current_asNeeded is True:
                    sex_list.append(0 if patient_sex == 'female' else 1)
                    age_list.append(patient_age)
                    weight_list.append(patient_weight)
                    frequency_list.append(-1)
                    period_list.append(-1)
                    periodUnit_list.append("-1")
                    asNeeded_list.append(True)
                    code_list.append(med_code)

                # else: skip entries that don't meet either condition


# Create DataFrame
data = pd.DataFrame({
    'sex': sex_list,
    'age': age_list,
    'weight': weight_list,
    'frequency': frequency_list,
    'period': period_list,
    'periodUnit_raw': periodUnit_list,
    'asNeeded': asNeeded_list,
    'code': code_list
})

# One-hot encode periodUnit_raw
periodUnit_dummies = pd.get_dummies(data['periodUnit_raw'], prefix='periodUnit')
data = pd.concat([data.drop('periodUnit_raw', axis=1), periodUnit_dummies], axis=1)

# remove redundat column
data = data.drop(columns=['periodUnit_-1'], errors='ignore')

data


,sex,age,weight,frequency,period,asNeeded,code,periodUnit_d,periodUnit_h
0,1,61,103.6,-1,-1.0,True,849574,False,False
1,1,61,103.6,4,1.0,False,1049625,True,False
2,1,61,103.6,1,3.0,False,245134,True,False
3,1,61,103.6,2,1.0,False,1860491,True,False
4,1,61,103.6,4,1.0,False,1049625,True,False
...,...,...,...,...,...,...,...,...,...
1229,1,54,124.9,-1,-1.0,True,310965,False,False
1230,0,6,20.0,-1,-1.0,True,665078,False,False
1231,0,6,20.0,-1,-1.0,True,1870230,False,False
1232,0,6,20.0,3,1.0,False,308192,True,False


When we add new data this might change

In [81]:
data["weight"].isna().sum()

np.int64(0)